In [6]:
import os, io, json, argparse, textwrap, time
from typing import Dict, Any
from pptx import Presentation
from pptx.util import Inches
from PIL import Image, ImageDraw, ImageFont
import jsonschema
import mimetypes
import urllib.parse
import urllib.request

In [8]:
def load_image_bytes(image_prompt: str) -> bytes:
    """
    Tries to interpret image_prompt as:
      1) URL -> download bytes
      2) local file path -> read bytes
      3) otherwise -> return a generated placeholder image
    """
    if not image_prompt:
        return generate_placeholder_image("No image_prompt provided.")

    # URL?
    try:
        parsed = urllib.parse.urlparse(image_prompt)
        if parsed.scheme in ("http", "https"):
            with urllib.request.urlopen(image_prompt, timeout=15) as resp:
                return resp.read()
    except Exception:
        pass

    # Local file?
    try:
        if os.path.exists(image_prompt):
            with open(image_prompt, "rb") as f:
                return f.read()
    except Exception:
        pass

    # Fallback: placeholder from text
    return generate_placeholder_image(image_prompt)


def generate_placeholder_image(caption: str) -> bytes:
    from PIL import Image, ImageDraw, ImageFont
    import io, textwrap

    W, H = 1280, 720
    img = Image.new("RGB", (W, H), (245, 245, 245))
    draw = ImageDraw.Draw(img)

    # Choose a font (fallback to default if truetype not available)
    try:
        font_title = ImageFont.truetype("arial.ttf", 40)
        font_body  = ImageFont.truetype("arial.ttf", 28)
    except Exception:
        font_title = ImageFont.load_default()
        font_body  = ImageFont.load_default()

    title = "IMAGE PLACEHOLDER"
    subtitle = (caption or "")[:240]

    def measure(text: str, font):
        """Return (width, height) using textbbox when available; fallback to textsize."""
        try:
            # Pillow ≥ 8.0: textbbox returns (x0, y0, x1, y1)
            x0, y0, x1, y1 = draw.textbbox((0, 0), text, font=font)
            return (x1 - x0, y1 - y0)
        except AttributeError:
            # Older Pillow
            return draw.textsize(text, font=font)

    # Draw title centered
    tw, th = measure(title, font_title)
    y = 200
    draw.text(((W - tw) // 2, y), title, font=font_title, fill=(60, 60, 60))
    y += th + 24

    # Wrap subtitle roughly to fit
    wrapped = textwrap.wrap(subtitle, width=46)
    for line in wrapped:
        lw, lh = measure(line, font_body)
        draw.text(((W - lw) // 2, y), line, font=font_body, fill=(80, 80, 80))
        y += lh + 8

    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()


def add_image_fitted(slide, img_bytes: bytes, left_in: float, top_in: float,
                     max_w_in: float, max_h_in: float):
    """
    Adds an image scaled to fit inside the given box (inches), maintaining aspect ratio.
    """
    from pptx.util import Inches
    im = Image.open(io.BytesIO(img_bytes))
    w_px, h_px = im.size
    ar = w_px / max(h_px, 1)

    # compute target size within bounds
    target_w_in = max_w_in
    target_h_in = target_w_in / max(ar, 0.0001)
    if target_h_in > max_h_in:  # too tall, limit by height
        target_h_in = max_h_in
        target_w_in = target_h_in * ar

    # center within the box horizontally if there's leftover space
    left = Inches(left_in + (max_w_in - target_w_in) / 2.0)
    top = Inches(top_in)
    slide.shapes.add_picture(
        io.BytesIO(img_bytes), left, top,
        width=Inches(target_w_in), height=Inches(target_h_in)
    )

In [ ]:
os.environ["DEEPSEEK_API_KEY"] = ""
#这个在发布github的版本里面建议删掉后面一串数字

In [11]:


# Try import OpenAI client (DeepSeek uses OpenAI-compatible API)
try:
    from openai import OpenAI
except Exception:
    OpenAI = None

# ---- JSON schema for slide_spec ----
SLIDE_SPEC_SCHEMA = {
    "type": "object",
    "required": ["title", "slides"],
    "properties": {
        "title": {"type": "string"},
        "slides": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["type", "title"],
                "properties": {
                    "type": {"type": "string", "enum": ["title_slide", "bullet_slide", "image_slide", "two_column"]},
                    "title": {"type": "string"},
                    "subtitle": {"type": "string"},
                    "bullets": {"type": "array", "items": {"type": "string"}},
                    "image_prompt": {"type": "string"},
                    "notes": {"type": "string"}
                }
            }
        }
    }
}

# ---- Helper: prompt -> call DeepSeek (OpenAI-compatible) ----
def call_deepseek_llm(user_prompt: str, model: str = "deepseek-chat", temperature: float = 0.2) -> Dict[str, Any]:
    """
    使用 OpenAI Python SDK（OpenAI class）调用 DeepSeek（需要 DEEPSEEK_API_KEY 环境变量）。
    返回解析后的 slide_spec dict。
    """
    if OpenAI is None:
        raise RuntimeError("openai package not available in this environment.")
    api_key = os.environ.get("DEEPSEEK_API_KEY")
    if not api_key:
        raise RuntimeError("环境变量 DEEPSEEK_API_KEY 未设置。")

    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
    system_prompt = (
        "You are a slide-authoring assistant. Given the user's request, output ONLY a valid JSON object matching the schema:\n"
        '{ "title": "string", "slides": [{"type":"title_slide|bullet_slide|image_slide|two_column", "title":"string", "subtitle?":"string", "bullets?":["..."], "image_prompt?":"string", "notes?":"string"}] }\n'
        "Constraints: max 10 slides. Bullets should be concise. Output STRICT JSON only (no markdown)."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    resp = client.chat.completions.create(model=model, temperature=temperature, messages=messages, stream=False)
    # try to extract JSON-like substring
    raw = resp.choices[0].message.content
    json_text = extract_json_like(raw)
    return json.loads(json_text)

def extract_json_like(s: str) -> str:
    """从模型的返回文本中抽取第一个 { ... } JSON 块；若失败则抛异常。"""
    s = s.strip()
    # remove triple-fence if present
    if s.startswith("```"):
        parts = s.split("```")
        for p in parts:
            p = p.strip()
            if p.startswith("{"):
                s = p
                break
    start = s.find("{")
    end = s.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("无法从模型输出中提取 JSON。原始输出片段:\n" + s[:1000])
    return s[start:end+1]


# ---- Helper: render PPT with python-pptx ----
def render_pptx_from_spec(spec: Dict[str, Any], out_path: str = "output.pptx"):
    """根据 slide_spec 生成 pptx 并保存"""
    # basic presentation (if you have a custom template, load it)
    prs = Presentation()

    for s in spec.get("slides", []):
        stype = s.get("type", "bullet_slide")
        if stype == "title_slide":
            layout = prs.slide_layouts[0] if len(prs.slide_layouts)>0 else prs.slide_layouts[1]
            slide = prs.slides.add_slide(layout)
            if slide.shapes.title:
                slide.shapes.title.text = s.get("title","")
            if s.get("subtitle"):
                slide.shapes.add_textbox(Inches(1), Inches(1.6), Inches(8), Inches(0.8)).text_frame.text = s.get("subtitle")

        elif stype == "bullet_slide":
            layout = prs.slide_layouts[1] if len(prs.slide_layouts)>1 else prs.slide_layouts[0]
            slide = prs.slides.add_slide(layout)
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass
            # body: prefer placeholder 1
            body_tf = None
            try:
                body_tf = slide.shapes.placeholders[1].text_frame
                body_tf.clear()
            except Exception:
                tb = slide.shapes.add_textbox(Inches(1), Inches(1.6), Inches(6), Inches(4))
                body_tf = tb.text_frame
                body_tf.clear()
            # bullets: avoid an empty first line
            #I also altered this part
            bullets = s.get("bullets", [])
            if bullets:
                body_tf.text = bullets[0]
                for b in bullets[1:]:
                    p = body_tf.add_paragraph(); p.text = b; p.level = 0

            # right-side image thumbnail (URL, local path, or placeholder)
            if s.get("image_prompt"):
                try:
                    img_bytes = load_image_bytes(s["image_prompt"])
                except Exception as e:
                    img_bytes = generate_placeholder_image(f'{s["image_prompt"]}\n(Load failed: {e})')

                # place in a 3.2in x 4.0in box at (left=6.0in, top=1.6in)
                add_image_fitted(
                    slide, img_bytes,
                    left_in=6.0, top_in=1.6,
                    max_w_in=3.2, max_h_in=4.0
                )
        elif stype == "two_column":
            layout = prs.slide_layouts[6] if len(prs.slide_layouts)>6 else prs.slide_layouts[1]
            slide = prs.slides.add_slide(layout)
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass
            left_tb = slide.shapes.add_textbox(Inches(0.6), Inches(1.6), Inches(4.2), Inches(4))
            left_tf = left_tb.text_frame
            left_tf.clear()
            for b in s.get("bullets", []):
                p = left_tf.add_paragraph()
                p.text = b
        # I changed this part Nov 6. The goal of this part is to 
        elif stype == "image_slide":
            slide = prs.slides.add_slide(prs.slide_layouts[6] if len(prs.slide_layouts)>6 else prs.slide_layouts[0])
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass

            prompt_or_path = s.get("image_prompt") or s.get("title") or "placeholder"
            try:
                img_bytes = load_image_bytes(prompt_or_path)
            except Exception as e:
                img_bytes = generate_placeholder_image(f"{prompt_or_path}\n(Load failed: {e})")

            # big hero image area (left, top, max_w, max_h in inches)
            add_image_fitted(slide, img_bytes, left_in=0.6, top_in=1.6, max_w_in=8.8, max_h_in=3.6)

            # optional caption bullets
            bullets = s.get("bullets", [])
            if bullets:
                tf = slide.shapes.add_textbox(Inches(0.6), Inches(5.5), Inches(8.8), Inches(1.2)).text_frame
                tf.clear()
                tf.text = bullets[0]
                for b in bullets[1:]:
                    p = tf.add_paragraph(); p.text = b
        else:
            # fallback simple text slide
            slide = prs.slides.add_slide(prs.slide_layouts[1] if len(prs.slide_layouts)>1 else prs.slide_layouts[0])
            try:
                slide.shapes.title.text = s.get("title","")
            except Exception:
                pass
            if s.get("notes"):
                tb = slide.shapes.add_textbox(Inches(1), Inches(1.6), Inches(8), Inches(2))
                tb.text_frame.text = s.get("notes","")

    prs.save(out_path)
    print(f"[DONE] Saved PPT: {out_path}")


# ---- Main orchestration ----
def main():
    p = argparse.ArgumentParser()
    p.add_argument("--prompt", type=str, required=False, default="", help="User prompt for PPT content")
    p.add_argument("--out", type=str, default="output.pptx", help="Output pptx path")
    p.add_argument("--model", type=str, default="deepseek-chat", help="Model name for DeepSeek")

    # Use parse_known_args to avoid ipykernel's extra args breaking argparse when running inside Jupyter
    args, unknown = p.parse_known_args()


    prompt = args.prompt.strip()
    if not prompt:
        # interactive if no prompt provided
        try:
            prompt = input("Enter a short prompt describing the PPT you want: ").strip()
        except Exception:
            prompt = ""
        if not prompt:
            print("No prompt provided, exiting.")
            return

    # choose LLM
    spec = None

    print("[INFO] Calling DeepSeek API to generate slide spec...")
    try:
        spec = call_deepseek_llm(prompt, model=args.model, temperature=0.2)
    except Exception as e:
        print("[WARN] DeepSeek call failed:", e)


    # Validate spec
    try:
        jsonschema.validate(instance=spec, schema=SLIDE_SPEC_SCHEMA)
    except Exception as e:
        print("[ERROR] slide_spec 验证失败:", e)
        print("slide_spec 内容:\n", json.dumps(spec, ensure_ascii=False, indent=2))

    # Render PPT (images are placeholder; replace generate_placeholder_image with real image API if desired)
    render_pptx_from_spec(spec, out_path=args.out)


if __name__ == "__main__":
    main()

[INFO] Calling DeepSeek API to generate slide spec...
[DONE] Saved PPT: output.pptx
